### Imports

In [ ]:
!pip install -q -U google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.4/725.4 kB 14.4 MB/s eta 0:00:00


In [ ]:
!pip install -q -U openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.0/375.0 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai
import numpy as np
import pandas as pd
import re
import copy
import requests
import time
from openai import OpenAI
from tqdm import tqdm
from requests.exceptions import HTTPError, Timeout, RequestException
from enum import Enum
import datetime
import os
import pytz

### Classes

In [ ]:
class Prompts(Enum):
    initial =  "The current experiment includes many trials. Your task, in each trial, is to select one of two options, A or B. Each selection will be followed by the presentation of both choice’s payoffs. Your payoff for the trial is the payoff of the selection you made. To begin, please select A or B. Respond only with the letter 'A' or 'B', indicating your choice. Do not provide an explanation, reason, or any additional text. If you respond with anything other than a single letter (A or B), the answer will be considered incorrect."
    initial_a = "The current experiment includes many trials. Your task, in each trial, is to select one of two options, A or B. Each selection will be followed by the presentation of both choice’s payoffs. Your payoff for the trial is the payoff of the selection you made."
    initial_b = "To begin, please select A or B. Respond only with the letter 'A' or 'B', indicating your choice.Do not provide an explanation, reason, or any additional text. If you respond with anything other than a single letter (A or B), the answer will be considered incorrect."
    cont = "Please make a new selection, responding only with the letter 'A' or 'B', indicating your choice. Respond with only the letter of your choice (A or B) and nothing else. Do not provide an explanation, reason, or any additional text. If you respond with anything other than a single letter (A or B), the answer will be considered incorrect."
    hist_dict = "The following summarizes the past rounds, where 'choice' is what you chose, 'outcome' is what you recieved, and 'alternative' is what you would have recieved, had you chosen the other option. The numbers represent which round this occured in."
    hist_chat = "The following is the chat history of the past rounds, where keys called 'User' indicate the chat prompts, and those entitled 'Agent' indicate your past choices in this trial."

    init_reasoning =  "The current experiment includes many trials. Your task, in each trial, is to select one of two options, A or B. Each selection will be followed by the presentation of both choice’s payoffs. Your payoff for the trial is the payoff of the selection you made. To begin, please select A or B. Your resposne must begin with the letter 'A' or 'B', indicating your choice, followed by a short motivation for your choice."
    cont_reasoning = "Please make a new selection, first responding with the letter 'A' or 'B', indicating your choice, followed by a short motivation for your choice. Respond first only the letter of your choice (A or B) and afterwards provide an explanation."

In [ ]:
class GeminiModel:
  def __init__(self, api_key: str, model_name: str = 'gemini-1.5-flash',  provide_history: (str | None) = None, temp: float = 1):
    genai.configure(api_key=api_key)
    self.model_name = model_name
    self.temp = temp
    self.generation_config = {"temperature": self.temp, "top_p": 0.95, "top_k": 64, "max_output_tokens": 8192, "response_mime_type": "text/plain"}
    self.model = genai.GenerativeModel(model_name = self.model_name, generation_config=self.generation_config)
    self.history_provided = provide_history
    self.messages = [{"role": "user", "parts": Prompts.initial.value}] if self.history_provided == 'chat' else Prompts.initial.value
    self.history = {} if self.history_provided else self.model.start_chat(history=[])

  def generate_response(self):
    return self.model.generate_content(self.messages) if self.history_provided else self.history.send_message(self.messages)

  def set_prompt(self, round_details: dict):
    round_num = max(round_details.keys())
    prev_round = round_details[round_num]
    prev_alt = 'B' if prev_round['choice'] == 'A' else 'A'
    if self.history_provided == 'all':
      self.messages = f"You selected {prev_round['choice']} and recieved a payoff of {prev_round['outcome']}. Had you selected {prev_alt}, you would have recieved {prev_round['alternative']}." + Prompts.hist_dict.value + f"{self.history}. The current round number is {round_num}:" + Prompts.cont.value
    elif self.history_provided == 'chat':
     self.messages.append({"role": "model", "parts": [prev_round['choice']]})
     self.messages.append({"role": "user", "parts": [f"You selected {prev_round['choice']} and recieved a payoff of {prev_round['outcome']}. Had you selected {prev_alt}, you would have recieved {prev_round['alternative']}." + Prompts.cont.value]})
    else:
      self.messages = f"You selected {prev_round['choice']} and recieved a payoff of {prev_round['outcome']}. Had you selected {prev_alt}, you would have recieved {prev_round['alternative']}." + Prompts.cont.value

  def set_history(self, round_details: dict):
    if self.history_provided == 'all':
        self.history = round_details

  def get_response(self):
    retries = 0
    while retries < 7:
      try:
        response = self.generate_response()
        r = re.sub(r'\s+', '', response.text)
        if r == 'A' or r == 'B':
          return r
        else:
          self.prompt = "Your previous response was invalid. Please ensure your choice is a single character, either 'A' or 'B', and nothing more." + self.prompt
          print('retrying call')
          continue
      except Exception as e:
        print(f"An error occurred: {e}")

      time.sleep(0.5 * (2 * retries))
      retries += 1

    print("Max retries exceeded. API call failed.")
    return None

  def get_model_dict(self):
    return {'model_name': self.model_name, 'history_provided': self.history_provided, 'temperature': self.temp}


In [ ]:
class OpenAIModel:
  def __init__(self, api_key: str, model_name: str = 'gpt-4o-mini',  provide_history: str = 'all', temp: float = 1):
    self.client = OpenAI(api_key=api_key)
    self.model_name = model_name
    self.temp = temp
    self.history_provided = provide_history
    self.messages = [{"role": "system", "content": Prompts.initial_a.value},
                   {"role": "user", "content": Prompts.initial_b.value}]
    self.history = {}

  def set_history(self, round_details: dict):
    if self.history_provided == 'all':
      self.history = round_details

  def set_prompt(self, round_details: dict):
    round_num = max(round_details.keys())
    prev_round = round_details[round_num]
    prev_alt = 'B' if prev_round['choice'] == 'A' else 'A'
    if self.history_provided == 'chat':
      self.messages.append({"role": "assistant", "content": prev_round['choice']})
      self.messages.append({"role": "user", "content": f"You selected {prev_round['choice']} and recieved a payoff of {prev_round['outcome']}. Had you selected {prev_alt}, you would have recieved {prev_round['alternative']}." + Prompts.cont.value})
    else:
      self.messages = [{"role": "system", "content": Prompts.initial_a.value},
                        {"role": "user", "content": f"You selected {prev_round['choice']} and recieved a payoff of {prev_round['outcome']}. Had you selected {prev_alt}, you would have recieved {prev_round['alternative']}." + Prompts.hist_dict.value + f"{self.history}. The current round number is {round_num}:" + Prompts.cont.value}]

  def get_response(self):
    retries = 0
    while retries < 7:
      try:
        response = self.client.chat.completions.create(model=self.model_name, messages=self.messages, temperature = self.temp)
        r = re.sub(r'\s+', '', response.choices[0].message.content)
        if r == 'A' or r == 'B':
          return r
        elif r[-1] == 'A' or r[-1] == 'B':
            return r[-1]
        else:
          print(self.messages)
          print(f'retrying call, given response:{r}')
          self.messages.append({"role": "assistant", "content": r})
          self.messages.append({"role": "user", "content": "Your previous response was invalid. Please ensure your choice is a single character, either 'A' or 'B', and nothing more."})
      except Exception as e:
        print(f"An error occurred: {e}")

      time.sleep(0.5 * (2 * retries))
      retries += 1

    print("Max retries exceeded. API call failed.")
    return None


  def get_model_dict(self):
    return {'model_name': self.model_name, 'history_provided': self.history_provided, 'temperature': self.temp}


In [ ]:
class Trial:
  def __init__(self, trial_id: int, trials_df: pd.DataFrame, model: (GeminiModel | OpenAIModel), num_rounds: int = 100, trial_idx: (str | int) = 'random'):
    idx = np.random.randint(0, len(trials_df)) if trial_idx == 'random' else trial_idx
    row = trials_df.iloc[idx]
    self.num_rounds = num_rounds
    self.trial_id = trial_id
    self.prob_id = row['id']
    self.a1 = row['a1']
    self.pa1 = row['pa1']
    self.a2 = row['a2']
    self.b1 = row['b1']
    self.pb1 = row['pb1']
    self.b2 = row['b2']
    self.corrAB = row['corrAB']
    self.model = model
    self.round_num = 1
    self.rounds_details = {}

  def create_trial_dict(self):
    self.trial_dict = {'trial_id': self.trial_id, 'prob_id': self.prob_id, 'a1': self.a1, 'pa1': self.pa1, 'a2': self.a2, 'b1': self.b1, 'pb1': self.pb1, 'b2': self.b2, 'corrAB': self.corrAB}
    self.trial_dict.update(self.model.get_model_dict())
    return self.trial_dict

  def create_results_df(self):
    d = self.create_trial_dict()
    rows = []
    for round_number, round_info in self.rounds_details.items():
      row = {**d, 'round_number': round_number, **round_info,}
      rows.append(row)
    return pd.DataFrame(rows)

  def run_trial(self):
    for i in tqdm(range(self.num_rounds)):
      choice = self.model.get_response()
      if choice == None:
        break
      self.simulate_round(choice)
      self.model.set_history(self.rounds_details)
      self.model.set_prompt(self.rounds_details)
      self.round_num += 1
    return self.create_results_df()

  def simulate_round(self, choice: str):
    rand_a, rand_b = np.random.rand(), np.random.rand()
    if self.corrAB != 0:
      rand_b = rand_a if self.corrAB == 1 else 1 - rand_a
    if choice == 'A':
        outcome = self.a1 if rand_a < self.pa1 else self.a2
        alternative = self.b1 if rand_b < self.pb1 else self.b2
    else:
        outcome = self.b1 if rand_b < self.pb1 else self.b2
        alternative = self.a1 if rand_a < self.pa1 else self.a2
    self.rounds_details[self.round_num] = {'choice': choice,
                                              'outcome': outcome,
                                              'alternative': alternative}

In [ ]:
def get_timestamp():
  gmt_plus_3 = pytz.timezone('Etc/GMT-3')
  now = datetime.datetime.now(gmt_plus_3)
  timestamp = now.strftime('%Y-%m-%d %H:%M')
  return timestamp

### Running in Colab

In [ ]:
from google.colab import userdata, files
gemini_token = userdata.get('gemini-rp')
openai_token = userdata.get('openai_rp')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path_prefix = '/content/drive/MyDrive/Archive/University/Research Project/'
df = pd.read_csv(os.path.join(path_prefix, 'experiment_problems.csv'))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
trial_params = {'idx': [0,1,2,3]}
model_params = {'model_history': [('gemini-1.5-flash', None), ('gemini-1.5-flash', 'all'), ('gemini-1.5-flash', 'chat'), ('gpt-4o-mini', 'all'), ('gpt-4o-mini', 'chat'), ('gpt-3.5-turbo', 'all'), ('gpt-3.5-turbo', 'chat')],
                'temperature': [0.2, 1, 2]}

In [ ]:
results_df = pd.DataFrame()
num_trials = 20
rounds = 100
all_temps = False

last_id = 1238
trial_idx = trial_params['idx'][0]
model_name = model_params['model_history'][5][0]
history = model_params['model_history'][5][1]
temp = model_params['temperature'][1]

In [ ]:
model_name, history, temp, last_id

('gpt-3.5-turbo', 'all', 1, 1238)

In [ ]:
results_df = pd.DataFrame()
if all_temps:
    for temp in model_params['temperature']:
      print(f'--------------- Running 20 {model_name, history} Trials at Temperature {temp} ---------------')
      timestamp = get_timestamp()
      for i in range(num_trials):
        model = GeminiModel(gemini_token, model_name, history, temp) if model_name == 'gemini-1.5-flash' else OpenAIModel(openai_token, model_name, history, temp)
        trial = Trial(i+last_id+1, df, model, rounds, trial_idx)
        trial_results = trial.run_trial()
        if len(trial_results) < 100:
          break
        results_df = pd.concat([results_df, trial_results])
        print(f'\n Trial {i+1}/{num_trials} Complete!')
        results_df.to_csv(os.path.join(path_prefix, f'trial_results_{model_name}_{history}_temp_{temp}_{timestamp}.csv'), index=False)
      last_id += 20
    results_df.to_csv(os.path.join(path_prefix, f'trial_results_{model_name}_{history}_{get_timestamp()}.csv'), index=False)
else:
    print(f'--------------- Running {num_trials} {model_name, history} Trials at Temperature {temp} ---------------')
    timestamp = get_timestamp()
    for i in range(num_trials):
      model = GeminiModel(gemini_token, model_name, history, temp) if model_name == 'gemini-1.5-flash' else OpenAIModel(openai_token, model_name, history, temp)
      trial = Trial(i+last_id+1, df, model, rounds, trial_idx)
      trial_results = trial.run_trial()
      if len(trial_results) < 100:
        break
      results_df = pd.concat([results_df, trial_results])
      print(f'\n Trial {i+1}/{num_trials} Complete!')
      results_df.to_csv(os.path.join(path_prefix, f'trial_results_{model_name}_{history}_temp_{temp}_{timestamp}.csv'), index=False)

--------------- Running 20 ('gpt-3.5-turbo', 'all') Trials at Temperature 1 ---------------


100%|██████████| 100/100 [00:43<00:00,  2.30it/s]



 Trial 1/20 Complete!


  0%|          | 0/100 [00:00<?, ?it/s]

[{'role': 'system', 'content': 'The current experiment includes many trials. Your task, in each trial, is to select one of two options, A or B. Each selection will be followed by the presentation of both choice’s payoffs. Your payoff for the trial is the payoff of the selection you made.'}, {'role': 'user', 'content': "To begin, please select A or B. Respond only with the letter 'A' or 'B', indicating your choice.Do not provide an explanation, reason, or any additional text. If you respond with anything other than a single letter (A or B), the answer will be considered incorrect."}]
retrying call, given response:PleaseselectAorB.
[{'role': 'system', 'content': 'The current experiment includes many trials. Your task, in each trial, is to select one of two options, A or B. Each selection will be followed by the presentation of both choice’s payoffs. Your payoff for the trial is the payoff of the selection you made.'}, {'role': 'user', 'content': "To begin, please select A or B. Respond o

  0%|          | 0/100 [00:26<?, ?it/s]

Max retries exceeded. API call failed.


In [ ]:
results_df

,trial_id,prob_id,a1,pa1,a2,b1,pb1,b2,corrAB,model_name,history_provided,temperature,round_number,choice,outcome,alternative
0,977,4.0,6.0,0.5,0.0,8.0,0.5,0.0,-1.0,gemini-1.5-flash,all,0.2,1,A,6.0,0.0
1,977,4.0,6.0,0.5,0.0,8.0,0.5,0.0,-1.0,gemini-1.5-flash,all,0.2,2,A,0.0,8.0
2,977,4.0,6.0,0.5,0.0,8.0,0.5,0.0,-1.0,gemini-1.5-flash,all,0.2,3,B,0.0,6.0
3,977,4.0,6.0,0.5,0.0,8.0,0.5,0.0,-1.0,gemini-1.5-flash,all,0.2,4,B,0.0,6.0
4,977,4.0,6.0,0.5,0.0,8.0,0.5,0.0,-1.0,gemini-1.5-flash,all,0.2,5,B,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1036,4.0,6.0,0.5,0.0,8.0,0.5,0.0,-1.0,gemini-1.5-flash,all,2.0,96,A,0.0,8.0
96,1036,4.0,6.0,0.5,0.0,8.0,0.5,0.0,-1.0,gemini-1.5-flash,all,2.0,97,B,0.0,6.0
97,1036,4.0,6.0,0.5,0.0,8.0,0.5,0.0,-1.0,gemini-1.5-flash,all,2.0,98,B,0.0,6.0
98,1036,4.0,6.0,0.5,0.0,8.0,0.5,0.0,-1.0,gemini-1.5-flash,all,2.0,99,B,0.0,6.0


### Motivation Experiments

In [ ]:
def generate_response(model, messages, history_provided, history):
  return model.generate_content(messages) if history_provided else history.send_message(messages)

def get_model_dict(model_name, history_provided, temp = 1):
    return {'model_name': model_name, 'history_provided': history_provided, 'temperature': temp}

def set_prompt(round_details: dict, history_provided, history):
  round_num = max(round_details.keys())
  prev_round = round_details[round_num]
  prev_alt = 'B' if prev_round['choice'] == 'A' else 'A'
  if history_provided == 'all':
    messages = f"You selected {prev_round['choice']} and recieved a payoff of {prev_round['outcome']}. Had you selected {prev_alt}, you would have recieved {prev_round['alternative']}." + Prompts.hist_dict.value + f"{history}. The current round number is {round_num}:" + Prompts.cont_reasoning.value
  elif history_provided == 'chat':
    messages.append({"role": "model", "parts": [prev_round['choice']]})
    messages.append({"role": "user", "parts": [f"You selected {prev_round['choice']} and recieved a payoff of {prev_round['outcome']}. Had you selected {prev_alt}, you would have recieved {prev_round['alternative']}." + Prompts.cont_reasoning.value]})
  else:
    messages = f"You selected {prev_round['choice']} and recieved a payoff of {prev_round['outcome']}. Had you selected {prev_alt}, you would have recieved {prev_round['alternative']}." + Prompts.cont_reasoning.value
  return messages

def set_history(history, history_provided, round_details: dict):
  if history_provided == 'all':
      history = round_details
  return history

def get_response(model, messages, history_provided, history):
  retries = 0
  while retries < 7:
    try:
      response = generate_response(model, messages, history_provided, history)
      print(response.text)
      #r = re.sub(r'\s+', '', response.text[:2])
      r = response.text
      if r[0] == 'A' or r[0] == 'B' or r[1] == 'A' or r[1] == 'B':
        return messages, r[0], r[2:]
      else:
        if history_provided == 'chat':
          messages.append({"role": "assistant", "parts": [r]})
          messages.append({"role": "user", "parts": "Your previous response was invalid. Please ensure your choice begins with single character, either 'A' or 'B', followed by your reasoning."})
        else:
          messages = "Your previous response was invalid. Please ensure your choice begins with single character, either 'A' or 'B', followed by your reasoning." + messages
        print('retrying call')
        continue
    except Exception as e:
      print(f"An error occurred: {e}")

    time.sleep(0.5 * (2 * retries))
    retries += 1

  print("Max retries exceeded. API call failed.")
  return None

In [ ]:
class ReasoningTrial:
  def __init__(self, trial_id: int, trials_df: pd.DataFrame, model: (GeminiModel | OpenAIModel), num_rounds: int = 100, trial_idx: (str | int) = 'random'):
    idx = np.random.randint(0, len(trials_df)) if trial_idx == 'random' else trial_idx
    row = trials_df.iloc[idx]
    self.num_rounds = num_rounds
    self.trial_id = trial_id
    self.prob_id = row['id']
    self.a1 = row['a1']
    self.pa1 = row['pa1']
    self.a2 = row['a2']
    self.b1 = row['b1']
    self.pb1 = row['pb1']
    self.b2 = row['b2']
    self.corrAB = row['corrAB']
    self.round_num = 1
    self.rounds_details = {}

  def create_trial_dict(self, model_name, history_provided):
    self.trial_dict = {'trial_id': self.trial_id, 'prob_id': self.prob_id, 'a1': self.a1, 'pa1': self.pa1, 'a2': self.a2, 'b1': self.b1, 'pb1': self.pb1, 'b2': self.b2, 'corrAB': self.corrAB}
    self.trial_dict.update(get_model_dict(model_name, history_provided))
    return self.trial_dict

  def create_results_df(self, model_name, history_provided):
    d = self.create_trial_dict(model_name, history_provided)
    rows = []
    for round_number, round_info in self.rounds_details.items():
      row = {**d, 'round_number': round_number, **round_info,}
      rows.append(row)
    return pd.DataFrame(rows)

  def run_trial(self, model, messages, history_provided, history):
    for i in tqdm(range(self.num_rounds)):
      messages, choice, reasoning = get_response(model, messages, history_provided, history)
      if choice == None:
        break
      self.simulate_round(choice, reasoning)
      history = set_history(history, history_provided, self.rounds_details)
      messages = set_prompt(self.rounds_details, history_provided, history)
      self.round_num += 1
    return self.create_results_df(model_name, history_provided)

  def simulate_round(self, choice: str, reasoning: str):
    rand_a, rand_b = np.random.rand(), np.random.rand()
    if self.corrAB != 0:
      rand_b = rand_a if self.corrAB == 1 else 1 - rand_a
    if choice == 'A':
        outcome = self.a1 if rand_a < self.pa1 else self.a2
        alternative = self.b1 if rand_b < self.pb1 else self.b2
    else:
        outcome = self.b1 if rand_b < self.pb1 else self.b2
        alternative = self.a1 if rand_a < self.pa1 else self.a2
    self.rounds_details[self.round_num] = {'choice': choice,
                                              'outcome': outcome,
                                              'alternative': alternative,
                                              'motivation': reasoning}

In [ ]:
model_name = 'gemini-1.5-flash'
history_provided = 'all'

genai.configure(api_key=gemini_token)
model = genai.GenerativeModel(model_name = model_name)
messages = [{"role": "user", "parts": Prompts.init_reasoning.value}] if history_provided == 'chat' else Prompts.init_reasoning.value
history = {} if history_provided else model.start_chat(history=[])

In [ ]:
trial = ReasoningTrial(100000, df, model, 100, 0)
trial_results = trial.run_trial(model, messages, history_provided, history)
timestamp = get_timestamp()
trial_results.to_csv(os.path.join(path_prefix, f'reasoning_trial_results_{model_name}_{history_provided}_{timestamp}.csv'), index=False)

  1%|          | 1/100 [00:12<20:27, 12.40s/it]

As an AI, I am unable to make choices based on subjective factors like "motivation" or "preference." I require specific information about the potential payoffs for option A and option B to provide a reasoned response. 

Please provide the payoffs for each option, and I will analyze them to determine the optimal choice. 



  2%|▏         | 2/100 [00:13<09:31,  5.83s/it]

B 
This is a simple case of maximizing reward. Since B would have yielded a higher payoff in the previous round, choosing B in the current round has a higher probability of yielding a better outcome. 



  3%|▎         | 3/100 [00:14<06:03,  3.75s/it]

...

B

The history of the game shows a very strong pattern of alternating outcomes between A and B. Since the last round resulted in a positive outcome for B, it's reasonable to assume that B might have a positive outcome in this round. 



 99%|█████████▉| 99/100 [02:18<00:01,  1.39s/it]

B

The history of the game shows a very strong pattern of alternating outcomes between A and B. Since the last round resulted in a positive outcome for B, it's reasonable to assume that B might have a positive outcome in this round. 



100%|██████████| 100/100 [02:19<00:00,  1.40s/it]

B

The history of the game shows a very strong pattern of alternating outcomes between A and B. Since the last round resulted in a positive outcome for B, it's reasonable to assume that B might have a positive outcome in this round. 



In [ ]:
trial_results

,trial_id,prob_id,a1,pa1,a2,b1,pb1,b2,corrAB,model_name,history_provided,temperature,round_number,choice,outcome,alternative,motivation
0,100000,1.0,0.0,1.0,0.0,1.0,0.9,-10.0,0.0,gemini-1.5-flash,None,1,1,A,0.0,1.0,"an AI, I don't have the ability to make choic..."
1,100000,1.0,0.0,1.0,0.0,1.0,0.9,-10.0,0.0,gemini-1.5-flash,None,1,2,B,1.0,0.0,\nThe previous round showed that option B yiel...
2,100000,1.0,0.0,1.0,0.0,1.0,0.9,-10.0,0.0,gemini-1.5-flash,None,1,3,B,1.0,0.0,\nOption B has consistently provided a higher ...
3,100000,1.0,0.0,1.0,0.0,1.0,0.9,-10.0,0.0,gemini-1.5-flash,None,1,4,B,1.0,0.0,\nGiven that option B has consistently yielded...
4,100000,1.0,0.0,1.0,0.0,1.0,0.9,-10.0,0.0,gemini-1.5-flash,None,1,5,B,-10.0,0.0,\nOption B has consistently produced a higher ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,100000,1.0,0.0,1.0,0.0,1.0,0.9,-10.0,0.0,gemini-1.5-flash,None,1,96,A,0.0,1.0,\nThe previous round showed a significant nega...
96,100000,1.0,0.0,1.0,0.0,1.0,0.9,-10.0,0.0,gemini-1.5-flash,None,1,97,B,1.0,0.0,\nWhile option B resulted in a significant los...
97,100000,1.0,0.0,1.0,0.0,1.0,0.9,-10.0,0.0,gemini-1.5-flash,None,1,98,B,1.0,0.0,\nThe previous round confirmed that option B i...
98,100000,1.0,0.0,1.0,0.0,1.0,0.9,-10.0,0.0,gemini-1.5-flash,None,1,99,B,1.0,0.0,\nOption B has consistently produced a higher ...
